In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install nltk>=3.4.5
!pip install gensim>=4.1.2
!pip install simpletransformers>=0.61.10
!pip install torch>=1.6.0 transformers>=4.11.3 sentencepiece
!pip install numpy requests nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action
import ast
import re
import copy
from random import random
import json

In [ ]:
%cd /content/drive/MyDrive/CS-7650-Project-main/mrqa/
#!sh data/download_data.sh

/content/drive/MyDrive/CS-7650-Project-main/mrqa


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 5.3 MB/s 
     |████████████████████████████████| 132 kB 40.1 MB/s 
     |████████████████████████████████| 10.2 MB 41.0 MB/s 
     |████████████████████████████████| 79 kB 4.1 MB/s 


In [ ]:
!gunzip "/content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/SQuAD_aug.jsonl.gz"
!gunzip "/content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/HotpotQA_aug.jsonl.gz"
!gunzip "/content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/NaturalQuestions_aug.jsonl.gz"

gzip: /content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/SQuAD_aug.jsonl.gz: No such file or directory
gzip: /content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/HotpotQA_aug.jsonl.gz: No such file or directory
gzip: /content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/NaturalQuestions_aug.jsonl.gz: No such file or directory


In [ ]:
with open("/content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/SQuAD.jsonl", 'r') as json_file:
    json_list = list(json_file)
for i in range(len(json_list)):
  json_list[i] = ast.literal_eval(json_list[i])
print(json_list[1])

{'id': '', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'qas': [{'answers': ['Saint Bernadette Soubirous'], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182', 'qid': '38cc2597b6624bd8af1e8ba7f693096f', 'question_tokens': [['To', 0], ['whom', 3], ['did', 8], ['the'

In [ ]:
aug = naw.SynonymAug(aug_src='wordnet')
augmentRatio = 0.1
splitOn = ['''"''', """'""", ",", ";", ".", "?", "!", " "]
for i in range(1, len(json_list)):
  if(random() < augmentRatio):
    newContext = aug.augment(json_list[i]['context'])[0]
    json_list.append(copy.copy(json_list[i]))
    json_list[-1]['context'] = [newContext]
    new_context_token = ""
    context_tokens = []
    start_id = 0
    for j in range(len(newContext)):
      char = newContext[j]
      if(char in splitOn):
        if(new_context_token != ""):
          context_tokens.append([new_context_token, start_id])
          if(char != " "):
            start_id = j
          else:
            start_id = j + 1
        new_context_token = ""
      if(char != " "):
        new_context_token += char
    if(new_context_token != ""):
      context_tokens.append([new_context_token, start_id])
    json_list[-1]['context_token'] = context_tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
with open("/content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/SQuAD_aug.jsonl", 'w') as fout:
    for entry in json_list:
        json.dump(entry, fout)
        fout.write('\n')

In [ ]:
!gzip "/content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/SQuAD_aug.jsonl"
!gzip "/content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/HotpotQA_aug.jsonl"
!gzip "/content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/NaturalQuestions_aug.jsonl"
!rm -rf "/content/drive/MyDrive/CS-7650-Project-main/mrqa/data/train/SQuAD.jsonl"

In [ ]:
!python main.py --epochs=2 --batch_size=8 --lr=3e-5 --do_lower_case --adv --use_cuda --dis_lambda=0.01

Streaming output truncated to the last 5000 lines.
CLS:  torch.Size([8, 768])
Sequence output:  torch.Size([8, 384, 768])
Sequence output:  torch.Size([8, 384, 768])
CLS:  torch.Size([8, 768])
Sequence output:  torch.Size([8, 384, 768])
Sequence output:  torch.Size([8, 384, 768])
CLS:  torch.Size([8, 768])
Sequence output:  torch.Size([8, 384, 768])
Sequence output:  torch.Size([8, 384, 768])
CLS:  torch.Size([8, 768])
Sequence output:  torch.Size([8, 384, 768])
Sequence output:  torch.Size([8, 384, 768])
CLS:  torch.Size([8, 768])
Sequence output:  torch.Size([8, 384, 768])
Sequence output:  torch.Size([8, 384, 768])
CLS:  torch.Size([8, 768])
Sequence output:  torch.Size([8, 384, 768])
Sequence output:  torch.Size([8, 384, 768])
CLS:  torch.Size([8, 768])
Sequence output:  torch.Size([8, 384, 768])
Sequence output:  torch.Size([8, 384, 768])
CLS:  torch.Size([8, 768])
Sequence output:  torch.Size([8, 384, 768])
Sequence output:  torch.Size([8, 384, 768])
CLS:  torch.Size([8, 768])
Se